### *Pulling NPI data using API from npi registry*

In [1]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize

In [ ]:
base_url = 'https://npiregistry.cms.hhs.gov/api/'

In [2]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize

base_url = 'https://npiregistry.cms.hhs.gov/api/'

def fetch_npi_data(npi):
    # Construct the URL for the API request
    #url = base_url + 'version=2.1'
    params = {
        'number': npi,
        'version': '2.1'
    }
    
    url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
    try:
        # Send the API request
        response = requests.get(url, params=params)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            # Process the data as per your requirements
            return data
        else:
            # Request was not successful, handle the error
            print('Error:', response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        # Request encountered an exception, handle the error
        print('Error:', str(e))
        return None


In [11]:
raw_input = pd.read_csv("C:/Users/GopinathPandit/OneDrive - ConcertAI/Desktop/test.csv", names= ['NPI'])

In [195]:
raw_data = pd.DataFrame()
    
    
address = pd.DataFrame()
p_address = pd.DataFrame()


for i in raw_input['NPI']:
    try:
        df = pd.json_normalize(fetch_npi_data(i)['results'])
#         for j in fetch_npi_data(i)['results'][0]['addresses']:
#             df2 = pd.json_normalize(fetch_npi_data(i)['results'][j],'addresses','number')
#             df3 = pd.json_normalize(fetch_npi_data(i)['results'][j],'practiceLocations','number')

#         address = pd.concat([address,df2])
#         p_address = pd.concat([p_address,df3])
        raw_data = pd.concat([raw_data,df])
    except Exception as e:
        print(e)


    

required_col = ['number','enumeration_type',
 'basic.first_name',
 'basic.last_name',
 'basic.middle_name',
 'basic.credential',
 'basic.sole_proprietor',
 'basic.gender',
 'basic.enumeration_date',
 'basic.last_updated',
 'basic.certification_date',
 'basic.status',
 'basic.name_prefix',
 'basic.name_suffix','addresses',
               'practiceLocations']


basic_data = raw_data[required_col]




primary_practice_location = pd.DataFrame()
for i in basic_data.number:

    df3 = pd.json_normalize(pd.json_normalize(basic_data[(basic_data.number == i)]['addresses'])[1])
    df3['npi'] = i
    
    primary_practice_location = pd.concat([primary_practice_location, df3])



secondary_practice_location_1 = pd.DataFrame()
for i in basic_data.number:
    try:
        df3 = pd.json_normalize(pd.json_normalize(basic_data[(basic_data.number == i)]['practiceLocations'])[0])
        df3['npi'] = i

        secondary_practice_location_1 = pd.concat([secondary_practice_location_1, df3])
    except:
        pass


secondary_practice_location_2 = pd.DataFrame()
for i in basic_data.number:
    try:
        df3 = pd.json_normalize(pd.json_normalize(basic_data[(basic_data.number == i)]['practiceLocations'])[1])
        df3['npi'] = i

        secondary_practice_location_2 = pd.concat([secondary_practice_location_2, df3])
    except:
        pass
    
#Droping the address and practiceLocations columns
    
basic_data = basic_data.drop(['addresses',
               'practiceLocations'], axis = 1)



#renaming the columns
new_columns = ['primary_' + col for col in primary_practice_location.columns]
primary_practice_location = primary_practice_location.rename(columns=dict(zip(primary_practice_location.columns, new_columns)))

new_columns = ['secondary_' + col + '_1' for col in secondary_practice_location_1.columns]
secondary_practice_location_1 = secondary_practice_location_1.rename(columns=dict(zip(secondary_practice_location_1.columns, new_columns)))


new_columns = ['secondary_' + col + '_2' for col in secondary_practice_location_2.columns]
secondary_practice_location_2 = secondary_practice_location_2.rename(columns=dict(zip(secondary_practice_location_2.columns, new_columns)))

final_output = basic_data.merge(primary_practice_location, left_on= 'number', right_on='primary_npi', how = 'left').merge(secondary_practice_location_1, left_on= 'number', right_on='secondary_npi_1', how = 'left').merge(secondary_practice_location_2, left_on= 'number', right_on='secondary_npi_2', how = 'left')


final_output = final_output.drop(['secondary_npi_2','secondary_npi_1'], axis = 1)

final_output = final_output.rename(columns= {'number' :'NPI','basic.first_name': 'first_name',
'basic.last_name': 'last_name',
'basic.middle_name': 'middle_name',
'basic.credential': 'credential',
'basic.sole_proprietor': 'sole_proprietor',
'basic.gender': 'gender',
'basic.enumeration_date': 'enumeration_date',
'basic.last_updated': 'last_updated',
'basic.certification_date': 'certification_date',
'basic.status': 'status',
'basic.name_prefix': 'name_prefix',
'basic.name_suffix': 'name_suffix'
})

In [197]:
final_output.to_csv("C:/Users/GopinathPandit/OneDrive - ConcertAI/Desktop/npi_data.csv", index= False)

In [198]:
final_output

,NPI,enumeration_type,first_name,last_name,middle_name,credential,sole_proprietor,gender,enumeration_date,last_updated,...,secondary_country_name_2,secondary_address_purpose_2,secondary_address_type_2,secondary_address_1_2,secondary_city_2,secondary_state_2,secondary_postal_code_2,secondary_telephone_number_2,secondary_fax_number_2,secondary_address_2_2
0,1083386239,NPI-1,EVA,GLIEBERMAN,NaN,NaN,NO,F,2021-10-05,2021-10-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1962548941,NPI-1,MICHELLE,COSTA,F,D.O.,NO,F,2007-01-29,2010-05-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1235573379,NPI-1,MEREDITH,CLARY,MARIE,M.D.,NO,F,2013-04-19,2023-02-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992181200,NPI-1,STACY,RAUSCH,NaN,CNP,NO,F,2015-08-10,2015-10-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1124101225,NPI-1,CARL,DITCH,M,M.D.,YES,M,2006-10-23,2009-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1164700092,NPI-1,GENA,BORGMAN,MARIE,APRN,NO,F,2011-07-26,2022-06-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1285279398,NPI-1,RACHAEL,FORTUNATO,NaN,NP,NO,F,2019-11-12,2020-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1356838700,NPI-2,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-20,2020-01-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1548218183,NPI-1,ROBERT,SCHMITZ,J,M.D.,NO,M,2006-05-05,2012-04-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1619488053,NPI-1,ANDREW,KESTER,NaN,NaN,NO,M,2017-10-12,2017-10-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
